In [11]:
import requests  
import json
from datetime import datetime
import pytz  # Make sure to install pytz if you haven't already
import time
import re

# API endpoints
arrival_url = "https://brycecanyonshuttle.com/Services/JSONPRelay.svc/GetStopArrivalTimes"
vehicle_points_url = 'https://brycecanyonshuttle.com/Services/JSONPRelay.svc/GetMapVehiclePoints?apiKey=8882812681&isPublicMap=true'

params = {
    'apiKey': '8882812681',
    'stopIds': '2',
    'version': '2'
}

# Function to fetch vehicle points
def get_vehicle_points():
    response = requests.get(vehicle_points_url)
    jsonp_data = response.text
    json_data = re.sub(r'^[^(]*\((.*)\)$', r'\1', jsonp_data)
    return json.loads(json_data)

# Function to fetch description for a specific vehicle ID
def get_vehicle_description(vehicle_id):
    url = f'https://brycecanyonshuttle.com/Services/JSONPRelay.svc/GetVehicleRouteStopEstimates?quantity=1&vehicleIdStrings={vehicle_id}'
    response = requests.get(url)
    jsonp_data = response.text
    json_data = re.sub(r'^[^(]*\((.*)\)$', r'\1', jsonp_data)

    # Parse the JSON data
    data = json.loads(json_data)
    
    # Check if the data contains estimates and extract the first description
    if isinstance(data, list) and len(data) > 0:
        estimates = data[0].get('Estimates', [])
        if len(estimates) > 0:
            description = estimates[0].get('Description', 'No Description available')
            return description
    return "No Description available"

# Function to fetch arrival times
def fetch_vehicle_arrival_times():
    response = requests.get(arrival_url, params=params)

    if response.status_code == 200:
        jsonp_data = response.text

        # Strip the JSONP callback wrapper
        if jsonp_data.startswith("callback(") and jsonp_data.endswith(")"):
            json_data = jsonp_data[len("callback("):-1]
        else:
            json_data = jsonp_data  # Just in case it's pure JSON

        # Load as JSON
        try:
            parsed_data = json.loads(json_data)

            # Extract arrival times for each vehicle
            vehicle_arrival_seconds = {}
            for stop in parsed_data:
                for time in stop['Times']:
                    vehicle_id = time['VehicleId']
                    if vehicle_id is not None:  # Check if VehicleId is not None
                        seconds_until_arrival = time['Seconds']  # Use the seconds field
                        # Store the minimum seconds until arrival for each vehicle ID
                        if vehicle_id not in vehicle_arrival_seconds:
                            vehicle_arrival_seconds[vehicle_id] = seconds_until_arrival
                        else:
                            vehicle_arrival_seconds[vehicle_id] = min(vehicle_arrival_seconds[vehicle_id], seconds_until_arrival)

            # Convert to a list of tuples and sort from least seconds to most
            all_vehicle_arrivals = [(vehicle_id, seconds) for vehicle_id, seconds in vehicle_arrival_seconds.items()]
            sorted_vehicle_arrivals = sorted(all_vehicle_arrivals, key=lambda x: x[1])

            return sorted_vehicle_arrivals

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
    else:
        print(f"Failed to retrieve data: {response.status_code}")
    return []

# Update every 30 seconds
try:
    while True:
        arrivals = fetch_vehicle_arrival_times()

        print("Currently Active Vehicles:")
        for vehicle_id, seconds in arrivals:
            minutes = seconds / 60  # Convert seconds to minutes
            current_location = get_vehicle_description(vehicle_id)  # Get the current location description
            print(f"Vehicle ID: {vehicle_id}, Minutes Until Arrival: {minutes:.2f}, Current Location: {current_location}")
        print("\n" + "-"*50 + "\n")  # Separator for clarity
        time.sleep(30)  # Wait for 30 seconds before the next update

except KeyboardInterrupt:
    print("Stopped by user.")


Minutes Until Vehicle Arrivals (Sorted from Least to Most):
Vehicle ID: 7, Minutes Until Arrival: -3.73, Current Location: Bryce Canyon Visitor Center (SB)
Vehicle ID: 5, Minutes Until Arrival: 11.25, Current Location: Shuttle Station
Vehicle ID: 4, Minutes Until Arrival: 26.25, Current Location: Inspiration Point
Vehicle ID: 3, Minutes Until Arrival: 41.25, Current Location: Shuttle Station

--------------------------------------------------

Minutes Until Vehicle Arrivals (Sorted from Least to Most):
Vehicle ID: 7, Minutes Until Arrival: -4.28, Current Location: Bryce Canyon Visitor Center (SB)
Vehicle ID: 5, Minutes Until Arrival: 10.70, Current Location: Shuttle Station
Vehicle ID: 4, Minutes Until Arrival: 25.70, Current Location: Inspiration Point
Vehicle ID: 3, Minutes Until Arrival: 40.70, Current Location: Shuttle Station

--------------------------------------------------

Minutes Until Vehicle Arrivals (Sorted from Least to Most):
Vehicle ID: 7, Minutes Until Arrival: -4.82,

In [14]:
import requests  
import json
from datetime import datetime
import pytz  # Make sure to install pytz if you haven't already
import time
import re

# API endpoints
arrival_url = "https://brycecanyonshuttle.com/Services/JSONPRelay.svc/GetStopArrivalTimes"
vehicle_points_url = 'https://brycecanyonshuttle.com/Services/JSONPRelay.svc/GetMapVehiclePoints?apiKey=8882812681&isPublicMap=true'
capacities_url = 'https://brycecanyonshuttle.com/Services/JSONPRelay.svc/GetVehicleCapacities'

params = {
    'apiKey': '8882812681',
    'stopIds': '2',
    'version': '2'
}

# Function to fetch vehicle points
def get_vehicle_points():
    response = requests.get(vehicle_points_url)
    jsonp_data = response.text
    json_data = re.sub(r'^[^(]*\((.*)\)$', r'\1', jsonp_data)
    return json.loads(json_data)

# Function to fetch description for a specific vehicle ID
def get_vehicle_description(vehicle_id):
    url = f'https://brycecanyonshuttle.com/Services/JSONPRelay.svc/GetVehicleRouteStopEstimates?quantity=1&vehicleIdStrings={vehicle_id}'
    response = requests.get(url)
    jsonp_data = response.text
    json_data = re.sub(r'^[^(]*\((.*)\)$', r'\1', jsonp_data)

    # Parse the JSON data
    data = json.loads(json_data)
    
    # Check if the data contains estimates and extract the first description
    if isinstance(data, list) and len(data) > 0:
        estimates = data[0].get('Estimates', [])
        if len(estimates) > 0:
            description = estimates[0].get('Description', 'No Description available')
            return description
    return "No Description available"

# Function to fetch arrival times
def fetch_vehicle_arrival_times():
    response = requests.get(arrival_url, params=params)

    if response.status_code == 200:
        jsonp_data = response.text

        # Strip the JSONP callback wrapper
        if jsonp_data.startswith("callback(") and jsonp_data.endswith(")"):
            json_data = jsonp_data[len("callback("):-1]
        else:
            json_data = jsonp_data  # Just in case it's pure JSON

        # Load as JSON
        try:
            parsed_data = json.loads(json_data)

            # Extract arrival times for each vehicle
            vehicle_arrival_seconds = {}
            for stop in parsed_data:
                for time in stop['Times']:
                    vehicle_id = time['VehicleId']
                    if vehicle_id is not None:  # Check if VehicleId is not None
                        seconds_until_arrival = time['Seconds']  # Use the seconds field
                        # Store the minimum seconds until arrival for each vehicle ID
                        if vehicle_id not in vehicle_arrival_seconds:
                            vehicle_arrival_seconds[vehicle_id] = seconds_until_arrival
                        else:
                            vehicle_arrival_seconds[vehicle_id] = min(vehicle_arrival_seconds[vehicle_id], seconds_until_arrival)

            # Convert to a list of tuples and sort from least seconds to most
            all_vehicle_arrivals = [(vehicle_id, seconds) for vehicle_id, seconds in vehicle_arrival_seconds.items()]
            sorted_vehicle_arrivals = sorted(all_vehicle_arrivals, key=lambda x: x[1])

            return sorted_vehicle_arrivals

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
    else:
        print(f"Failed to retrieve data: {response.status_code}")
    return []

# Function to fetch vehicle capacities
def fetch_vehicle_capacities():
    response = requests.get(capacities_url)
    if response.status_code == 200:
        jsonp_data = response.text
        json_data = re.sub(r'^[^(]*\((.*)\)$', r'\1', jsonp_data)
        return json.loads(json_data)
    else:
        print(f"Failed to retrieve capacities: {response.status_code}")
    return []

# Update every 30 seconds
try:
    while True:
        arrivals = fetch_vehicle_arrival_times()
        capacities = fetch_vehicle_capacities()
        
        # Create a mapping of Vehicle ID to Capacity information
        capacity_map = {cap['VehicleID']: cap for cap in capacities}

        print("Currently Active Vehicles:")
        for vehicle_id, seconds in arrivals:
            minutes = seconds / 60  # Convert seconds to minutes
            current_location = get_vehicle_description(vehicle_id)  # Get the current location description
            
            # Get capacity info for the vehicle ID
            capacity_info = capacity_map.get(vehicle_id, {})
            capacity = capacity_info.get('Capacity', 'N/A')
            current_occupation = capacity_info.get('CurrentOccupation', 'N/A')
            percentage = capacity_info.get('Percentage', 'N/A')

            print(f"Vehicle ID: {vehicle_id}, Minutes Until Arrival: {minutes:.2f}, Current Location: {current_location}, "
                  f"Capacity: {capacity}, Current Occupation: {current_occupation}, Percentage: {percentage:.2%}")

        print("\n" + "-"*50 + "\n")  # Separator for clarity
        time.sleep(30)  # Wait for 30 seconds before the next update

except KeyboardInterrupt:
    print("Stopped by user.")


Currently Active Vehicles:
Vehicle ID: 5, Minutes Until Arrival: 2.60, Current Location: Ruby's Campground (SB), Capacity: 40, Current Occupation: 2, Percentage: 5.00%
Vehicle ID: 2, Minutes Until Arrival: 17.60, Current Location: Shuttle Station, Capacity: 40, Current Occupation: 0, Percentage: 0.00%
Vehicle ID: 3, Minutes Until Arrival: 32.60, Current Location: Shuttle Station, Capacity: 40, Current Occupation: 0, Percentage: 0.00%
Vehicle ID: 7, Minutes Until Arrival: 47.60, Current Location: Bryce Point, Capacity: 40, Current Occupation: 4, Percentage: 10.00%

--------------------------------------------------

Stopped by user.


In [16]:
import requests  
import json
from datetime import datetime
import pytz  # Make sure to install pytz if you haven't already
import time
import re

# API endpoints
arrival_url = "https://brycecanyonshuttle.com/Services/JSONPRelay.svc/GetStopArrivalTimes"
vehicle_points_url = 'https://brycecanyonshuttle.com/Services/JSONPRelay.svc/GetMapVehiclePoints?apiKey=8882812681&isPublicMap=true'
capacities_url = 'https://brycecanyonshuttle.com/Services/JSONPRelay.svc/GetVehicleCapacities'

params = {
    'apiKey': '8882812681',
    'stopIds': '2',
    'version': '2'
}

# Function to fetch vehicle points
def get_vehicle_points():
    response = requests.get(vehicle_points_url)
    jsonp_data = response.text
    json_data = re.sub(r'^[^(]*\((.*)\)$', r'\1', jsonp_data)
    return json.loads(json_data)

# Function to fetch description for a specific vehicle ID
def get_vehicle_description(vehicle_id):
    url = f'https://brycecanyonshuttle.com/Services/JSONPRelay.svc/GetVehicleRouteStopEstimates?quantity=1&vehicleIdStrings={vehicle_id}'
    response = requests.get(url)
    jsonp_data = response.text
    json_data = re.sub(r'^[^(]*\((.*)\)$', r'\1', jsonp_data)

    # Parse the JSON data
    data = json.loads(json_data)
    
    # Check if the data contains estimates and extract the first description
    if isinstance(data, list) and len(data) > 0:
        estimates = data[0].get('Estimates', [])
        if len(estimates) > 0:
            description = estimates[0].get('Description', 'No Description available')
            return description
    return "No Description available"

# Function to fetch arrival times
def fetch_vehicle_arrival_times():
    response = requests.get(arrival_url, params=params)

    if response.status_code == 200:
        jsonp_data = response.text

        # Strip the JSONP callback wrapper
        if jsonp_data.startswith("callback(") and jsonp_data.endswith(")"):
            json_data = jsonp_data[len("callback("):-1]
        else:
            json_data = jsonp_data  # Just in case it's pure JSON

        # Load as JSON
        try:
            parsed_data = json.loads(json_data)

            # Extract arrival times for each vehicle
            vehicle_arrival_seconds = {}
            for stop in parsed_data:
                for time in stop['Times']:
                    vehicle_id = time['VehicleId']
                    if vehicle_id is not None:  # Check if VehicleId is not None
                        seconds_until_arrival = time['Seconds']  # Use the seconds field
                        # Store the minimum seconds until arrival for each vehicle ID
                        if vehicle_id not in vehicle_arrival_seconds:
                            vehicle_arrival_seconds[vehicle_id] = seconds_until_arrival
                        else:
                            vehicle_arrival_seconds[vehicle_id] = min(vehicle_arrival_seconds[vehicle_id], seconds_until_arrival)

            # Convert to a list of tuples and sort from least seconds to most
            all_vehicle_arrivals = [(vehicle_id, seconds) for vehicle_id, seconds in vehicle_arrival_seconds.items()]
            sorted_vehicle_arrivals = sorted(all_vehicle_arrivals, key=lambda x: x[1])

            return sorted_vehicle_arrivals

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
    else:
        print(f"Failed to retrieve data: {response.status_code}")
    return []

# Function to fetch vehicle capacities
def fetch_vehicle_capacities():
    response = requests.get(capacities_url)
    if response.status_code == 200:
        jsonp_data = response.text
        json_data = re.sub(r'^[^(]*\((.*)\)$', r'\1', jsonp_data)
        return json.loads(json_data)
    else:
        print(f"Failed to retrieve capacities: {response.status_code}")
    return []

# Update every 30 seconds
try:
    while True:
        arrivals = fetch_vehicle_arrival_times()
        capacities = fetch_vehicle_capacities()
        
        # Create a mapping of Vehicle ID to Capacity information
        capacity_map = {cap['VehicleID']: cap for cap in capacities}

        print("Currently Active Vehicles:")
        for vehicle_id, seconds in arrivals:
            minutes = seconds / 60  # Convert seconds to minutes
            current_location = get_vehicle_description(vehicle_id)  # Get the current location description
            
            # Get current percentage for the vehicle ID
            capacity_info = capacity_map.get(vehicle_id, {})
            percentage = capacity_info.get('Percentage', 0)  # Default to 0 if not found

            print(f"Vehicle ID: {vehicle_id}, Minutes Until Arrival: {minutes:.2f}, Current Location: {current_location}, "
                  f"Current Occupancy: {percentage:.2%}")

        print("\n" + "-"*50 + "\n")  # Separator for clarity
        time.sleep(15)  # Wait for some seconds before the next update

except KeyboardInterrupt:
    print("Stopped by user.")


Currently Active Vehicles:
Vehicle ID: 5, Minutes Until Arrival: 0.67, Current Location: Bryce Canyon Visitor Center (SB), Current Occupancy: 10.00%
Vehicle ID: 2, Minutes Until Arrival: 15.67, Current Location: Shuttle Station, Current Occupancy: 0.00%
Vehicle ID: 3, Minutes Until Arrival: 30.67, Current Location: Shuttle Station, Current Occupancy: 0.00%
Vehicle ID: 7, Minutes Until Arrival: 45.67, Current Location: Bryce Point, Current Occupancy: 27.50%

--------------------------------------------------

Currently Active Vehicles:
Vehicle ID: 5, Minutes Until Arrival: 0.37, Current Location: Bryce Canyon Visitor Center (SB), Current Occupancy: 10.00%
Vehicle ID: 2, Minutes Until Arrival: 15.37, Current Location: Shuttle Station, Current Occupancy: 0.00%
Vehicle ID: 3, Minutes Until Arrival: 30.37, Current Location: Shuttle Station, Current Occupancy: 0.00%
Vehicle ID: 7, Minutes Until Arrival: 45.37, Current Location: Bryce Point, Current Occupancy: 27.50%

------------------------

Currently Active Vehicles:
Vehicle ID: 5, Minutes Until Arrival: 0.15, Current Location: Bryce Canyon Visitor Center (SB), Current Occupancy: 10.00%
Vehicle ID: 2, Minutes Until Arrival: 10.95, Current Location: Shuttle Station, Current Occupancy: 0.00%
Vehicle ID: 3, Minutes Until Arrival: 25.95, Current Location: Shuttle Station, Current Occupancy: 0.00%
Vehicle ID: 7, Minutes Until Arrival: 40.95, Current Location: Inspiration Point, Current Occupancy: 57.50%

--------------------------------------------------

Currently Active Vehicles:
Vehicle ID: 5, Minutes Until Arrival: 0.15, Current Location: Bryce Canyon Visitor Center (SB), Current Occupancy: 10.00%
Vehicle ID: 2, Minutes Until Arrival: 10.65, Current Location: Shuttle Station, Current Occupancy: 0.00%
Vehicle ID: 3, Minutes Until Arrival: 25.65, Current Location: Shuttle Station, Current Occupancy: 0.00%
Vehicle ID: 7, Minutes Until Arrival: 40.65, Current Location: Inspiration Point, Current Occupancy: 57.50%

------------